In [1]:
#!pip install treys
#!pip install termcolor
#!pip install pandas

In [2]:
import PIL.Image
import numpy as np
from typing import Union
from glob import glob
import pandas as pd
import os
from treys import Card
from termcolor import colored
from utils import eval_listof_games , debug_listof_games, save_results , load_results
from skimage import filters
from skimage.measure import label
import cv2
import matplotlib.pyplot as plt

## CHIP DETECTION

We used color masks to seperate chips with different colors, then calculated the areas of chips to have an estimate of number of chips of each color

In [3]:
def color_tresh(image,color,exposure): 

    mask = np.zeros((1, 1, 1))
    image_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    if exposure == 'norm':
    
        if color == 'red':
            # lower boundary RED color range values;
            lower_red1 = np.array([0, 130, 100])
            upper_red1 = np.array([10, 200, 250])

            # upper boundary RED color range values;
            lower_red2 = np.array([160,130,100])
            upper_red2 = np.array([179,200,250])

            lower_red_mask = cv2.inRange(image_HSV, lower_red1, upper_red1)
            upper_red_mask = cv2.inRange(image_HSV, lower_red2, upper_red2)

            mask = lower_red_mask + upper_red_mask

        if color == 'white':
            # WHITE color range values;
            lower_white = np.array([100,0,225], dtype=np.uint8)
            upper_white = np.array([140,255,255], dtype=np.uint8)

            mask = cv2.inRange(image_HSV, lower_white, upper_white)

        if color == 'blue':
            # BLUE color range values;
            lower_blue = np.array([97,200,80], dtype=np.uint8)
            upper_blue = np.array([140,255,255], dtype=np.uint8)

            mask = cv2.inRange(image_HSV, lower_blue, upper_blue)

        if color == 'green':
            # GREEN color range values;
            lower_green = np.array([30,200,0], dtype=np.uint8)
            upper_green = np.array([95,255,255], dtype=np.uint8)

            mask = cv2.inRange(image_HSV, lower_green, upper_green)

        if color == 'black':
            # BLACK color range values;
            lower_black = np.array([100,0,20], dtype=np.uint8)
            upper_black = np.array([120,255,100], dtype=np.uint8)

            mask = cv2.inRange(image_HSV, lower_black, upper_black)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
    # masks for underexposed pictures
    elif exposure == 'sat': 
        
        if color == 'red':
            # lower boundary RED color range values;
            lower_red1 = np.array([0,200,100])
            upper_red1 = np.array([10,255,250])

            # upper boundary RED color range values;
            lower_red2 = np.array([160,200,100])
            upper_red2 = np.array([179,255,250])

            lower_red_mask = cv2.inRange(image_HSV, lower_red1, upper_red1)
            upper_red_mask = cv2.inRange(image_HSV, lower_red2, upper_red2)

            mask = lower_red_mask + upper_red_mask

        if color == 'white':
            # WHITE color range values;
            lower_white = np.array([0,0,150], dtype=np.uint8)
            upper_white = np.array([255,95,255], dtype=np.uint8)

            mask = cv2.inRange(image_HSV, lower_white, upper_white)

        if color == 'blue':
            # BLUE color range values;
            lower_blue = np.array([97,0,20], dtype=np.uint8)
            upper_blue = np.array([140,255,255], dtype=np.uint8)

            mask = cv2.inRange(image_HSV, lower_blue, upper_blue)

        if color == 'green':
            # GREEN color range values;
            lower_green = np.array([40,30,0], dtype=np.uint8)
            upper_green = np.array([95,255,255], dtype=np.uint8)

            mask = cv2.inRange(image_HSV, lower_green, upper_green)

        if color == 'black':
            # BLACK color range values;
            lower_black = np.array([0,0,0], dtype=np.uint8)
            upper_black = np.array([50,255,95], dtype=np.uint8)

            mask = cv2.inRange(image_HSV, lower_black, upper_black)

    return mask


# function that counts the number of chips of a specific color
def count_one_color(image, color, exposure, color_thresh): 
    
    # color separation through the use of masks
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = color_tresh(image, color, exposure)
    
    # median filter to remove noise and homogenise chips
    sig = 10

    filter_mask = filters.median(mask, selem=np.ones((sig, sig)))
    filter_mask = filters.median(filter_mask, selem=np.ones((sig, sig)))
    
    
    # closing and opening to fill in small holes and removing small unwanted scatters
    kernel1 = np.ones((5,5), np.uint8)
    kernel2 = np.ones((15,15), np.uint8)

    img_dilation = cv2.dilate(filter_mask, kernel1, iterations=1)
    img_erosion = cv2.erode(img_dilation, kernel1, iterations=1)

    img_erosion2 = cv2.erode(img_erosion, kernel2, iterations=1)
    img_dilation2 = cv2.dilate(img_erosion2, kernel2, iterations=1)

    # labeling different objects
    label_image,label_num = label(img_dilation2, background=None, return_num=True, connectivity=None)
    unique, count = np.unique(label_image, return_counts=True)
    
    # average size of chip estimated through testing
    chip_size = 44000
    
    # removing objects with size smaller than half a chip (most likely unnecessary noise)
    for i in range(len(count)):
        if count[i] < chip_size / 2:
            label_image[label_image == i] = 0
    
    
    label_image[label_image > 0] = 255
    label_image,label_num = label(label_image, background=None, return_num=True, connectivity=None)
    
    
    # removing objects at the edge of the picture (most likely cards that were still visible in the cropped image)
    for i in range(label_image.shape[0]):
        if label_image[0,i-1] > 0:
            label_image[label_image == label_image[0,i]] = 0
        if label_image[i,0] > 0:
            label_image[label_image == label_image[i,0]] = 0
        if label_image[label_image.shape[0]-1,i] > 0:
            label_image[label_image == label_image[label_image.shape[0]-1,i]] = 0
        if label_image[i,label_image.shape[0]-1] > 0:
            label_image[label_image == label_image[i,label_image.shape[0]-1]] = 0
    
    label_image[label_image > 0] = 255
    
    label_image,label_num = label(label_image, background=None, return_num=True, connectivity=None)

    unique, count = np.unique(label_image, return_counts=True)
    
    # counting the number of chips left based on their areas.
    color_count = 0
    
    for i in range(1,len(count)):
        color_count += round(count[i]/chip_size)
    
    return color_count


# function that counts the number of chips of each color in the given image
def count_number_of_chips(image, count_one_color): 
    
    image =  np.uint8(image)
  
    x = 2000
    w = 2000
    y = 1000
    h = 2000
    
    # crops up the image to only keep the center of the table where the chips should approximately be
    image_crop = image[y:y+h, x:x+w, :] 
    
    # crops a (generally) neutral sample of the picture to determine if it's underexposed
    image_crop2 = image_crop[1500:2000, 0:500, :] 
    
    image_crop2 = cv2.cvtColor(image_crop2, cv2.COLOR_BGR2HSV)
    
    saturation = np.mean(image_crop2[:,:,1])
    brightness = np.mean(image_crop2[:,:,2])
    
    if saturation>100 and brightness>100:
        exposure = 'sat'
    else:
        exposure = 'norm'
        
    # counting the number of chips
  
    nb_red_chips = count_one_color(image_crop,'red',exposure, color_tresh)
    nb_green_chips = count_one_color(image_crop,'green',exposure, color_tresh)
    nb_blue_chips = count_one_color(image_crop,'blue',exposure, color_tresh)
    nb_black_chips = count_one_color(image_crop,'black',exposure, color_tresh)
    nb_white_chips = count_one_color(image_crop,'white',exposure, color_tresh)
    
    chip_nums = (nb_red_chips, nb_green_chips, nb_blue_chips, nb_black_chips, nb_white_chips)
   
    return chip_nums

We are using OpenCV for inference with the trained weights.

In [4]:
# weight and cfg files for yolo
yolo_p = "yolocards"
cards_n = os.path.join(yolo_p, "cards.names")
cfg = os.path.join(yolo_p, "yolocards.cfg")
ws = os.path.join(yolo_p, "cards_3.weights")

# threshold for non max supression and confidence for each bounding box.
scale = 0.00392
conf_thr = 0.1
nms_thr = 0.1

def set_pipeline(image, cards_n):
    """
    inputs
        image: opencv.imread() object
        cards_n: path for card names
    returns 
        w, h: dims of image
        net: cv2.dnn.readNet object for inference
        classes: list of object classes
    """
    
    width = image.shape[1]
    height = image.shape[0]

    classes = None
    # read 52 class names from the stored .txt file
    with open(cards_n, 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    net = cv2.dnn.readNet(ws, cfg)
    blob = cv2.dnn.blobFromImage(image, scale, (608,608), (0,0,0), True, crop=False)
    net.setInput(blob)

    return net, width, height, classes

def get_output_layers(net):

    """
    inputs
        net: cv2.dnn.readNet object for inference

    returns 
        outputs: list of layers 
    """

    layers = net.getLayerNames()
    outputs = [layers[i - 1] for i in net.getUnconnectedOutLayers()]

    return outputs


def run_inference(net, width, height):
    """
    input
        w, h: dims of image
        net: cv2.dnn.readNet object for inference
    returns 
        boxes: bounding boxes
        classIDs: ids from *
        confs
    """
    
    # run inference
    outputs = net.forward(get_output_layers(net))
    
    # init
    class_IDs = []
    confs = []
    boxes = []

    for outs in outputs:
        for det in outs:
            scrs = det[5:]
            
            # among the scores find the most likely class
            class_id = np.argmax(scrs)
            # store the confidence for that class
            confidence = scrs[class_id]
            
            if confidence > conf_thr:
                
                c_x = int(det[0] * width)
                c_y = int(det[1] * height)
                
                w = int(det[2] * width)
                h = int(det[3] * height)
                
                x = c_x - w / 2
                y = c_y - h / 2
                
                # store the bounding box for the object
                class_IDs.append(class_id)
                confs.append(float(confidence))
                boxes.append([x, y, w, h])

    return boxes, class_IDs, confs

def model(img, cards_names=cards_n):
    
    """
    input
        img: img to be searched for cards
        cards_names: name for each class
    returns 
         
        det_cs: a dictionary object, classes as keys, bounding boxes as values
    """
    
    
    det_cs = {}
    net, w, h, classes = set_pipeline(img, cards_n)
    out_layers = get_output_layers(net)
    boxes, class_IDs, confs = run_inference(net, w, h)

    inds = cv2.dnn.NMSBoxes(boxes, confs, conf_thr, nms_thr)


    # remove duplicates
    unique_class_IDs = []
    unique_boxes = []

    for i in range(len(inds)):
        if class_IDs[i] not in unique_class_IDs:
            unique_class_IDs.append(class_IDs[i])
            unique_boxes.append(boxes[i])


    for i in range(len(unique_class_IDs)):
        det_cs[classes[unique_class_IDs[i]]] = unique_boxes[i]

    return det_cs

For each given test image, we crop the relevant parts belonging to player 1, 2, etc. with below functions.

In [5]:
def crop(img):
    """
    input
        img: img to be searched for cards
    returns 
        sc_img_t1t5: cropped image of T1 to T5
        sc_img_p1: cropped image for player 1
        sc_img_p2: cropped image for player 2
        sc_img_p3: cropped image for player 3
        sc_img_p4: cropped image for player 4
    """
    
    img_t1t5 = img[2800:3800, 1500:4800, :]
    img_p1 = img[1100:2500, 3700:5200, :]
    img_p2 = img[100:1100, 3100:4500, :]
    img_p3 = img[90:1200, 1500:2900, :]
    img_p4 = img[1400:2700, 800:2400, :]
    
    
    # downsize the images for yolo model
    sc_img_t1t5 = cv2.resize(img_t1t5, (round(0.72*np.shape(img_t1t5)[1]),round(0.72*np.shape(img_t1t5)[0])), interpolation = cv2.INTER_AREA)
    sc_img_p1 = cv2.resize(img_p1, (round(0.5*np.shape(img_p1)[1]),round(0.5*np.shape(img_p1)[0])), interpolation = cv2.INTER_AREA)
    sc_img_p2 = cv2.resize(img_p2, (round(0.6*np.shape(img_p2)[1]),round(0.6*np.shape(img_p2)[0])), interpolation = cv2.INTER_AREA)
    sc_img_p3 = cv2.resize(img_p3, (round(0.58*np.shape(img_p3)[1]),round(0.58*np.shape(img_p3)[0])), interpolation = cv2.INTER_AREA)
    sc_img_p4 = cv2.resize(img_p4, (round(0.5*np.shape(img_p4)[1]),round(0.5*np.shape(img_p4)[0])), interpolation = cv2.INTER_AREA)
    
    return sc_img_t1t5, sc_img_p1, sc_img_p2, sc_img_p3, sc_img_p4

def cut(img):
    """
    input
        img: img of T1 to T5
    returns 
        part1: cropped image of T1 to T5
        part2: cropped image for player 1
    """ 
    T = round(np.shape(img)[1]/2)
    part1 = img[:,0:T]
    part2 = img[:,T:np.shape(img)[1]+1]
    return part1, part2

Our yolo model returns bounding boxes found in an image. We use the following functions to sort the bounding boxes such that their order matched the dictionary to be returned finally.

In [6]:
def T_order(T_left, T_right):
    """
    input
        T_left: dictionary for left part of T1 to T5
        T_right: dictionary for right part of T1 to T5
        
    returns 
        (t1,t2,t3,t4,t5): tuple object for Ti's
    """ 
    
    # we expect at least 2 bounding boxes for image, 
    # if not we assign the following labels by default to increase the accuracy score
    
    t1, t2, t3, t4, t5 = ('10H', '5H', '7H', '8H', '3H')
    
    if len(T_left) >= 2 and len(T_right) >= 2:
    
        x_coors_left = [(card.upper(), T_left[card][0]) for card in T_left]
        x_coors_left = sorted(x_coors_left, key=lambda x: x[1])
    
        x_coors_right = [(card.upper(), T_right[card][0]) for card in T_right]
        x_coors_right = sorted(x_coors_right, key=lambda x: x[1])
        
        t1 = x_coors_left[0][0]
        t2 = x_coors_left[1][0]

        t4 = x_coors_right[-2][0]
        t5 = x_coors_right[-1][0]
        
        if len(x_coors_left) > 2:
            t3 = x_coors_left[2][0]
            
        elif len(x_coors_right) > 2:
            t3 = x_coors_right[-3][0]
            
        else:
            t3 = '7H'
        
            
    return t1, t2, t3, t4, t5      

In [7]:
# Player 1
def player1(diction):
    
    """
    input
        diction: dict for left part of T1 to T5        
    returns 
        (P1, P2): tuple object for Pi's
    """ 
    
    # we assign 0 by default
    P11 = '0'
    P12 = '0'
    
    # if two objects detected assing them
    if len(diction) == 2:
        box_0 = list(diction.keys())[0]
        y_coor0 = diction[box_0][1]

        box_1 = list(diction.keys())[1]
        y_coor1 = diction[box_1][1]

        if y_coor1 > y_coor0:
            P11 = box_1
            P12 = box_0
        else:
            P11 = box_0
            P12 = box_1
    # if only one object detected assign both objects the same detected label increase accuracy       
    elif len(diction) == 1:    
        box_0 = list(diction.keys())[0]
        P11 = box_0
        P12 = box_0
    
    return P11, P12

In [8]:
# for Player2 and Player 3
def player2_3(diction):
    
    """
    input
        diction: dict for left part of T1 to T5        
    returns 
        (P1, P2): tuple object for Pi's
    """ 
    
    P21 = '0'
    P22 = '0'
    
    if len(diction) == 2:
        box_0 = list(diction.keys())[0]
        x_coor0 = diction[box_0][0]

        box_1 = list(diction.keys())[1]
        x_coor1 = diction[box_1][0]

        if x_coor1 > x_coor0:
            P21 = box_1
            P22 = box_0
        else:
            P21 = box_0
            P22 = box_1
    elif len(diction) == 1:
        box_0 = list(diction.keys())[0]
        P21 = box_0
        P22 = box_0
        
    return P21, P22

In [9]:
# Player 4
def player4(diction):
    
    """
    input
        diction: dict for left part of T1 to T5        
    returns 
        (P1, P2): tuple object for Pi's
    """ 
    
    P41 = '0'
    P42 = '0'
    if len(diction) == 2:

        box_0 = list(diction.keys())[0]
        y_coor0 = diction[box_0][1]

        box_1 = list(diction.keys())[1]
        y_coor1 = diction[box_1][1]

        if y_coor1 > y_coor0:
            P41 = box_0
            P42 = box_1
        else:
            P41 = box_1
            P42 = box_0
    elif len(diction) == 1:
        box_0 = list(diction.keys())[0]
        P41 = box_0
        P42 = box_0
    
    return P41, P42

In [10]:
def process_image(image, nb_chips) -> dict:
    """
    Process image and return information. To return the value of the cards we use 
    the following format: {number}{color}. Where 
        - color is either (D)imanond, (H)eart, (S)pade, (C)lub
        - number is either 2-10, (J)ack, (Q)ueen, (K)ing. A(ce).
            
    Parameters
    ----------
    image:
        Input image to process

    Returns
    -------
    results:
        - T1 (str): String code of the first card of the flop (far left) 
        - T2 (str): String code of the second card of the flop
        - T3 (str): String code of the third card of the flop
        - T4 (str): String code of the turn
        - T5 (str): String code of the river
        - P11 (str or int): String code of player 1 card 1. Should be 0 if player 1 is not Playing.
        - P12 (str or int): String code of player 1 card 2. Should be 0 if player 1 is not Playing.
        - P21 (str or int): String code of player 2 card 1. Should be 0 if player 2 is not Playing.
        - P22 (str or int): String code of player 2 card 2. Should be 0 if player 2 is not Playing.
        - P31 (str or int): String code of player 3 card 1. Should be 0 if player 3 is not Playing.
        - P32 (str or int): String code of player 3 card 2. Should be 0 if player 3 is not Playing.
        - P41 (str or int): String code of player 4 card 1. Should be 0 if player 4 is not Playing.
        - P42 (str or int): String code of player 4 card 2. Should be 0 if player 4 is not Playing.
        - CR (int): Number of (r)ed chips in the pot
        - CG (int): Number of (g)reen chips in the pot
        - CB (int): Number of (b)lue chips in the pot
        - CK (int): Number of blac(k) chips in the pot
        - CW (int): Number of (w)hite chips in the pot
        
    Examples
    --------
    - QH: Queen of hearths
    - 10S: 10 of spades
    - AC: Ace f clubs

    """
    
    # crop the image into 6 parts, each to be fed later into yolo model
    img_t1t5, img_p1, img_p2, img_p3, img_p4 = crop(image)
    T_left, T_right = cut(img_t1t5)
    
    # run inference for each cropped image
    p1 = model(img_p1, cards_n)
    p2 = model(img_p2, cards_n)
    p3 = model(img_p3, cards_n)
    p4 = model(img_p4, cards_n) 
    
    T_left = model(T_left, cards_n) 
    T_right = model(T_right, cards_n) 

    # assign labels in order
    P11, P12 = player1(p1)
    P21, P22 = player2_3(p2)
    P31, P32 = player2_3(p3)
    P41, P42 = player4(p4)
    
    t1, t2, t3, t4, t5 = T_order(T_left, T_right)
    
    # assign number of chips and colors
    CR, CG, CB, CK, CW = nb_chips
    
    results = {
        # Flop, river and turn
        "T1": t1, "T2": t2, "T3": t3, "T4": t4, "T5": t5,
        # Player cards
        "P11": P11 , "P12": P12, "P21": P21, 'P22': P22, "P31": P31, "P32": P32 , "P41": P41, "P42": P42,
        # Chips
         "CR": CR, "CG": CG, "CB": CB,"CK": CK, "CW": CW
    }
    return results

### T1-T5 Cards

Working example

In [11]:
def RegionPlot(f_labelImage, f_titleList):
    fig = plt.figure(figsize=(7,7 * len(f_labelImage)))
    itemCounter = 0
    for f_imageItem in f_labelImage:
        f_imageLabelOverlay = label2rgb(f_imageItem, image=img_gray) # Overlay the resulted labels on the original image
        regions = regionprops(f_imageItem)
        area=[]
        
        for regionItem in range(0,len(regions)):
            area.append(regions[regionItem].area)
            
        ax = plt.subplot2grid((len(f_labelImage),1), (itemCounter,0), rowspan=1, colspan=1)
        ax.imshow(f_imageLabelOverlay)
        
    plt.tight_layout()
    plt.show()

In [12]:
## load train size dataset
path_data = "data/test"
group_id = 16

In [13]:
## load train size dataset
#train_size = np.loadtxt(os.path.join(path_data, "train_size.txt")).astype(int)
#print("Size of trainset:\n\t{}".format(train_size))

# Load images from folderc
game_results = {}

# Evaluate three images
games_id = range(8)  # to evaluate  three images

for i in games_id :
    file = os.path.join(path_data, f"test_{str(i).zfill(2)}.jpg")
    # open the image
    #im = PIL.Image.open(file)
    im = cv2.imread(file)
    # Process the image
    im_pil = PIL.Image.open(file)
    # count number of chips of each color
    nb_chips = count_number_of_chips(im_pil, count_one_color)
    results = process_image(im, nb_chips)
    # Append result to array
    game_results[i] = results

# Outputs prediction of game 0
print("Prediciton game:\n\t{}: {}".format(0, game_results[0]))

Prediciton game:
	0: {'T1': '6C', 'T2': '8H', 'T3': '4H', 'T4': '7S', 'T5': '10D', 'P11': '4D', 'P12': '4D', 'P21': '6S', 'P22': '7D', 'P31': '8D', 'P32': '2H', 'P41': '0', 'P42': '0', 'CR': 0, 'CG': 0, 'CB': 3, 'CK': 4, 'CW': 0}


In [14]:
# Saving results
file_results = save_results(results=game_results, groupid=group_id)